## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-12-10-18-56-49 +0000,nyt,FDA Expands Covid Vaccine Inquiry to Adult Deaths,https://www.nytimes.com/2025/12/09/health/fda-...
1,2025-12-10-18-50-20 +0000,nyt,"Louvre Heist Was Caught on Film, Officials Say...",https://www.nytimes.com/2025/12/10/world/europ...
2,2025-12-10-18-46-00 +0000,wsj,Opinion | Trump’s Blue-Slip Blues,https://www.wsj.com/opinion/trumps-blue-slip-b...
3,2025-12-10-18-43-44 +0000,nyt,Netflix and Paramount Battle for Warner Bros. ...,https://www.nytimes.com/2025/12/09/business/me...
4,2025-12-10-18-43-42 +0000,nyt,What Could Thwart the Texas G.O.P. From Pickin...,https://www.nytimes.com/2025/12/10/us/texas-ho...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2345/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
15,trump,53
45,new,18
140,out,16
134,fed,14
253,media,14


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
162,2025-12-10-11-56-00 +0000,wsj,The next Fed chair will have to walk the tight...,https://www.wsj.com/politics/policy/can-the-ne...,133
214,2025-12-10-09-00-00 +0000,cbc,Is Trump’s stark new security strategy the end...,https://www.cbc.ca/news/world/trump-world-orde...,111
262,2025-12-10-02-08-00 +0000,wsj,President Trump touted the economy as he embar...,https://www.wsj.com/politics/policy/trump-tout...,106
267,2025-12-10-01-42-00 +0000,wsj,Eileen Higgins became the first Democrat to wi...,https://www.wsj.com/politics/elections/democra...,106
68,2025-12-10-17-08-07 +0000,nypost,Hilarious moment Trump interrupted by Air Forc...,https://nypost.com/2025/12/10/us-news/trump-bu...,101


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
162,133,2025-12-10-11-56-00 +0000,wsj,The next Fed chair will have to walk the tight...,https://www.wsj.com/politics/policy/can-the-ne...
234,74,2025-12-10-05-05-52 +0000,bbc,Kids locked out of social media apps after Aus...,https://www.bbc.com/news/articles/cgl6gkd7pz6o...
111,67,2025-12-10-15-39-00 +0000,wsj,"New York City Comptroller Brad Lander, a progr...",https://www.wsj.com/politics/policy/nyc-comptr...
252,61,2025-12-10-03-16-48 +0000,cbc,'Venezuela will breathe again': María Corina M...,https://www.cbc.ca/news/world/nobel-peace-priz...
177,58,2025-12-10-11-03-45 +0000,nypost,DOJ civil rights chief Harmeet Dhillon reveals...,https://nypost.com/2025/12/10/us-news/harmeet-...
34,42,2025-12-10-17-46-00 +0000,wsj,Judge Orders Trump Administration to End Natio...,https://www.wsj.com/us-news/law/judge-orders-t...
272,41,2025-12-10-01-14-00 +0000,wsj,Kevin Hassett says he would rely on his own ju...,https://www.wsj.com/economy/central-banking/ke...
314,41,2025-12-09-22-33-02 +0000,nypost,"Netflix, Paramount shares sink as Wall Street ...",https://nypost.com/2025/12/09/media/netflix-pa...
360,40,2025-12-09-19-30-00 +0000,wsj,Nvidia AI Chips to Undergo Unusual U.S. Securi...,https://www.wsj.com/tech/ai/nvidia-ai-chips-to...
363,39,2025-12-09-19-12-00 +0000,wsj,"Bruce Blakeman, a Long Island Republican who t...",https://www.wsj.com/politics/elections/long-is...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
